# Detectando Fumadores - Selección y Entrenamiento del Modelo

En este caso nos encontramos con un problema de clasificación binario ya que queremos detectar, según los biomarcadores, si los sujetos son fumadores o no para poder realizar intervenciones preventivas de promoción de la salud.

## Importando Bibliotecas

In [5]:
# Bibliotecas básicas de análisis de datos
import numpy as np
import pandas as pd

# Bibliotecas de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Herramientas de preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Métricas de evaluación
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

# Para balancear clases desbalanceadas
from imblearn.over_sampling import SMOTE

# Configuración de visualización
%matplotlib inline
sns.set_style("whitegrid")

print('Bibliotecas importadas correctamente')

Bibliotecas importadas correctamente


## Carga de Datos

Con el fin de preparar un único script para la ejecución del modelo, vamos a cargar los datos en bruto y con el conocimiento adquirido en el análisis realizado en el notebook anterior vamos a optimizar la limpieza y la adecuación de los datos en el siguiente apartado.

In [6]:
train = pd.read_csv('../data/raw/train.csv')
test =pd.read_csv('../data/raw/test.csv')
print('Archivos cargados')

Archivos cargados


## Limpieza y adecuación de datos